# Project: Wrangling and Analyze Data

##### Import required libraries

In [838]:
import pandas as pd
import numpy as np
import requests
import tweepy
from tweepy import OAuthHandler
import json 
from timeit import default_timer as timer

# Set some Pandas viewing options
pd.set_option('max_rows', None)
pd.set_option('max_columns', None)
pd.set_option('max_colwidth', None)

## Data Gathering
In the cell below, gather **all** three pieces of data for this project and load them in the notebook. **Note:** the methods required to gather each data are different.
1. Directly download the WeRateDogs Twitter archive data (twitter_archive_enhanced.csv)

In [839]:
# Download twitter-archive-enhanced file directly from web and add to project.
# Import file contents into pandas dataframe called twitter_archive

twitter_archive = pd.read_csv('twitter-archive-enhanced.csv')

2. Use the Requests library to download the tweet image prediction (image_predictions.tsv)

In [840]:
# Import image_predictions.tsv using requests library

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
r = requests.get(url, allow_redirects=True)

# Save to local tsv file
open('image-predictions.tsv', 'wb').write(r.content)

# Load image-predictions into a dataframe
image_predtictions = pd.read_csv('image-predictions.tsv', sep='\t')

# https://www.tutorialspoint.com/downloading-files-from-web-using-python

3. Use the Tweepy library to query additional data via the Twitter API (tweet_json.txt)

In [841]:
# Note to evaluator - Not using the Twitter API since it does not work without having a paid developer account.
# Using the backup option of importing tweet_json.txt file directly from Udacity file in next code block.

# DO NOT RUN THIS CODE - adding failsafe so that this block will not run unless exec_api_code is set to True
exec_api_code = False

if exec_api_code:
    # Query Twitter API for each tweet in the Twitter archive and save JSON in a text file
    # These are hidden to comply with Twitter's API terms and conditions
    consumer_key = 'HIDDEN'
    consumer_secret = 'HIDDEN'
    access_token = 'HIDDEN'
    access_secret = 'HIDDEN'

    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)

    api = tweepy.API(auth, wait_on_rate_limit=True)

    # NOTE TO STUDENT WITH MOBILE VERIFICATION ISSUES:
    # df_1 is a DataFrame with the twitter_archive_enhanced.csv file. You may have to
    # change line 17 to match the name of your DataFrame with twitter_archive_enhanced.csv
    # NOTE TO REVIEWER: this student had mobile verification issues so the following
    # Twitter API code was sent to this student from a Udacity instructor
    # Tweet IDs for which to gather additional data via Twitter's API
    tweet_ids = twitter_archive.tweet_id.values
    len(tweet_ids)

    # Query Twitter's API for JSON data for each tweet ID in the Twitter archive
    count = 0
    fails_dict = {}
    start = timer()
    # Save each tweet's returned JSON as a new line in a .txt file
    with open('tweet_json.txt', 'w') as outfile:
        # This loop will likely take 20-30 minutes to run because of Twitter's rate limit
        for tweet_id in tweet_ids:
            count += 1
            print(str(count) + ": " + str(tweet_id))
            try:
                tweet = api.get_status(tweet_id, tweet_mode='extended')
                print("Success")
                json.dump(tweet._json, outfile)
                outfile.write('\n')
            except tweepy.TweepError as e:
                print("Fail")
                fails_dict[tweet_id] = e
                pass
    end = timer()
    print(end - start)
    print(fails_dict)

In [842]:
# Read the data from the tweet_json.txt file, line by line, into Pandas dataframe
with open('tweet_json.txt') as f:
    tweet_api = pd.DataFrame(json.loads(line) for line in f)

# Keep only the columns we need
tweet_details = tweet_api[['id', 'retweet_count', 'favorite_count']]

# https://stackoverflow.com/questions/20037430/reading-multiple-json-records-into-a-pandas-dataframe

## Assessing Data
In this section, detect and document at least **eight (8) quality issues and two (2) tidiness issue**. You must use **both** visual assessment
programmatic assessement to assess the data.

**Note:** pay attention to the following key points when you access the data.

* You only want original ratings (no retweets) that have images. Though there are 5000+ tweets in the dataset, not all are dog ratings and some are retweets.
* Assessing and cleaning the entire dataset completely would require a lot of time, and is not necessary to practice and demonstrate your skills in data wrangling. Therefore, the requirements of this project are only to assess and clean at least 8 quality issues and at least 2 tidiness issues in this dataset.
* The fact that the rating numerators are greater than the denominators does not need to be cleaned. This [unique rating system](http://knowyourmeme.com/memes/theyre-good-dogs-brent) is a big part of the popularity of WeRateDogs.
* You do not need to gather the tweets beyond August 1st, 2017. You can, but note that you won't be able to gather the image predictions for these tweets since you don't have access to the algorithm used.



#### Twitter Archive

In [843]:
twitter_archive.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643555336193/photo/1,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421306343426/photo/1,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181378084864/photo/1,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557279858688/photo/1,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",NaN,NaN,NaN,"https://twitter.com/dog_rates/status/891327558926688256/photo/1,https://twitter.com/dog_rates/status/891327558926688256/photo/1",12,10,Franklin,None,None,None,None


In [844]:
twitter_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [845]:
twitter_archive.name.value_counts()

None              745
a                  55
Charlie            12
Lucy               11
Oliver             11
Cooper             11
Lola               10
Penny              10
Tucker             10
Winston             9
Bo                  9
the                 8
Sadie               8
an                  7
Buddy               7
Bailey              7
Daisy               7
Toby                7
Scout               6
Jax                 6
Koda                6
Bella               6
Stanley             6
Milo                6
Dave                6
Jack                6
Rusty               6
Leo                 6
Oscar               6
Alfie               5
very                5
George              5
Bentley             5
Louis               5
Phil                5
Sunny               5
Oakley              5
Sammy               5
Gus                 5
Chester             5
Larry               5
Finn                5
Hank                4
Sampson             4
Chip                4
Jerry     

In [846]:
# There are Denominator values that are not equal to 10. Might be extracted incorrectly.
twitter_archive.query('rating_denominator != 10')[['text', 'rating_numerator', 'rating_denominator']]

,text,rating_numerator,rating_denominator
313,"@jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho",960,0
342,@docmisterio account started on 11/15/15,11,15
433,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
516,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24,7
784,"RT @dog_rates: After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https:/…",9,11
902,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165,150
1068,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9,11
1120,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204,170
1165,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
1202,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50


In [847]:
twitter_archive.text.sample(5)

2171                             This is Kirk. He just saw a bacon wrapped tennis ball and literally died. So sad. 12/10 RIP Kirk https://t.co/0twoigv5jP
151     This is Zooey. She's the world's biggest fan of illiterate delivery people. 13/10 not your fault they don't listen, Zooey https://t.co/ixOFQ1tfqE
1484         This is Bodie. He's not proud of what he did, but it needed to be done. 9/10 eight days was a pretty good streak tbh https://t.co/bpZsGMqVVP
1030                          This is Lenox. She's in a wheelbarrow. Silly doggo. You don't belong there. 10/10 would push around https://t.co/oYbVR4nBsR
1620           This is Oreo. She's a photographer and a model. Living a double pupple life. 12/10 such talent much cute would pet https://t.co/zNeLxJeAoL
Name: text, dtype: object

#### Twitter Details

In [848]:
tweet_details.head()

,id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


In [849]:
tweet_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   id              2354 non-null   int64
 1   retweet_count   2354 non-null   int64
 2   favorite_count  2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


In [850]:
tweet_details.describe()

,id,retweet_count,favorite_count
count,2.354000e+03,2354.000000,2354.000000
mean,7.426978e+17,3164.797366,8080.968564
std,6.852812e+16,5284.770364,11814.771334
min,6.660209e+17,0.000000,0.000000
25%,6.783975e+17,624.500000,1415.000000
50%,7.194596e+17,1473.500000,3603.500000
75%,7.993058e+17,3652.000000,10122.250000
max,8.924206e+17,79515.000000,132810.000000


#### Image Predictions

In [851]:
image_predtictions.head()

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [852]:
image_predtictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [853]:
image_predtictions.p1.sample(25)

882                         Samoyed
1950               Lakeland_terrier
1412           Bernese_mountain_dog
1530                     schipperke
81                       toy_poodle
127                        Pembroke
1935                 French_bulldog
792                          kuvasz
1299                          swing
902     West_Highland_white_terrier
372                          wombat
894                golden_retriever
2044                            pug
1367                         paddle
1377                       malamute
154        Chesapeake_Bay_retriever
1668               golden_retriever
1145                     Eskimo_dog
1134                   window_shade
1968                        whippet
1506             miniature_pinscher
1928               golden_retriever
333                             pug
1337       Chesapeake_Bay_retriever
10                 golden_retriever
Name: p1, dtype: object

In [854]:
image_predtictions.p2.sample(25)

945                         Pomeranian
287           Chesapeake_Bay_retriever
480                   golden_retriever
583                             beagle
518                             borzoi
822             Welsh_springer_spaniel
1322                           volcano
216     American_Staffordshire_terrier
2066                      Irish_setter
994                          Chihuahua
1218    American_Staffordshire_terrier
1095                        Eskimo_dog
322                          Chihuahua
396                Rhodesian_ridgeback
1048                           Samoyed
12                    English_foxhound
2047                    Great_Pyrenees
360                           Cardigan
1969                    Great_Pyrenees
589                    Irish_wolfhound
1407                Labrador_retriever
2059                            borzoi
1182                  Blenheim_spaniel
941                             kuvasz
761                         guinea_pig
Name: p2, dtype: object

In [855]:
image_predtictions.p3.sample(25)

239                     Cardigan
530           Labrador_retriever
1804                     doormat
1747                    papillon
866                       weasel
218                   Eskimo_dog
1879             Norfolk_terrier
1596                    malamute
1363             German_shepherd
116                       collie
258                   brown_bear
1770          Labrador_retriever
1066              French_bulldog
730                Gordon_setter
301                  bathing_cap
227               cocker_spaniel
1476              French_bulldog
1248           Shetland_sheepdog
1765                      cougar
1016                    umbrella
719                     Pekinese
1792              English_setter
25                       hamster
2034                      kelpie
1085    Chesapeake_Bay_retriever
Name: p3, dtype: object

### Quality issues
#### twitter_archive
1. Timestamp column is not in timestamp format
2. Column floofer name should be floof not floofer
3. Retweet messages should not be included, only original tweets. Columns observed: retweeted_status_id and in_reply_to_status_id
4. Columns contain missing values and are not needed for analysis. expanded_urls, in_reply_to_status_id, in_reply_to_user_id, retweeted_status_user_id, retweeted_status_id ,retweeted_status_timestamp 
5. Columns have inconsistent case, invalid names. Columns observed: name, doggo, floof, pupper, puppo. Also fix case p1, p2, p3 in image_predictions.
6. Values above and below 10 observed in rating_denominator column.
<br>
### tweet_details
7. Rename 'id' column to tweet_id
<br>
### image_predtictions
8. Non-Dog names in columns p1, p2, p3

### Tidiness issues
1. Dog varibles doggo, floofer (floof), pupper, and puppo are in individual columns in twitter_archive and should be categorical
2. Columns from image_predictions and tweet_details should merge with twitter_archive

## Cleaning Data
In this section, clean **all** of the issues you documented while assessing. 

**Note:** Make a copy of the original data before cleaning. Cleaning includes merging individual pieces of data according to the rules of [tidy data](https://cran.r-project.org/web/packages/tidyr/vignettes/tidy-data.html). The result should be a high-quality and tidy master pandas DataFrame (or DataFrames, if appropriate).

In [856]:
# Make copies of original pieces of data
twitter_archive_clean = twitter_archive.copy()
tweet_details_clean = tweet_details.copy()
image_pred_clean = image_predtictions.copy()

## Quality Issues
### Issue #1: Timestamp column is not in timestamp format

#### Define:
- In the twitter_archive_clean dataframe, change the dtype of the timestamp column to a pandas datetime. 

#### Code

In [857]:
twitter_archive_clean['timestamp'] = pd.to_datetime(twitter_archive_clean['timestamp'])

#### Test

In [858]:
twitter_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype              
---  ------                      --------------  -----              
 0   tweet_id                    2356 non-null   int64              
 1   in_reply_to_status_id       78 non-null     float64            
 2   in_reply_to_user_id         78 non-null     float64            
 3   timestamp                   2356 non-null   datetime64[ns, UTC]
 4   source                      2356 non-null   object             
 5   text                        2356 non-null   object             
 6   retweeted_status_id         181 non-null    float64            
 7   retweeted_status_user_id    181 non-null    float64            
 8   retweeted_status_timestamp  181 non-null    object             
 9   expanded_urls               2297 non-null   object             
 10  rating_numerator            2356 non-null   int64           

### Issue #2: Column floofer name should be floof not floofer

#### Define
- In twitter_archive_clean rename column floofer to floof

#### Code

In [859]:
twitter_archive_clean.rename(columns={'floofer' : 'floof'}, inplace=True)

#### Test

In [860]:
list(twitter_archive_clean)

['tweet_id',
 'in_reply_to_status_id',
 'in_reply_to_user_id',
 'timestamp',
 'source',
 'text',
 'retweeted_status_id',
 'retweeted_status_user_id',
 'retweeted_status_timestamp',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floof',
 'pupper',
 'puppo']

### Issue #3: Retweet messages should not be included, only original tweets. Columns observed: retweeted_status_id and in_reply_to_status_id: 

#### Define
- Remove all rows that are retweets or replies using retweeted_status_id and in_reply_to_status_id columns.

#### Code

In [861]:
# Find the index of the retweets
retweets = twitter_archive_clean[pd.notnull(twitter_archive_clean['retweeted_status_id'])].index

# Find the index of the replys
replys = twitter_archive_clean[pd.notnull(twitter_archive_clean['in_reply_to_status_id'])].index

# Drop the rows
twitter_archive_clean.drop(index=retweets, inplace=True)
twitter_archive_clean.drop(index=replys, inplace=True)

#### Test

In [862]:
# Confirm rows were deleted. Started with 2356 rows.
twitter_archive_clean.shape

(2097, 17)

### Issue #4: Columns contain missing values and are not needed for analysis. expanded_urls, in_reply_to_status_id, in_reply_to_user_id, retweeted_status_user_id, retweeted_status_id ,retweeted_status_timestamp

#### Define
- Drop expanded_urls, in_reply_to_status_id, in_reply_to_user_id, retweeted_status_user_id, retweeted_status_id ,retweeted_status_timestamp columns from twitter_archive_clean

#### Code

In [863]:
# Drop the columns
to_drop = ['expanded_urls', 'in_reply_to_status_id', 'in_reply_to_user_id', 'retweeted_status_user_id', 'retweeted_status_id' ,'retweeted_status_timestamp']
twitter_archive_clean.drop(to_drop, axis=1, inplace=True)

#### Test

In [864]:
list(twitter_archive_clean)

['tweet_id',
 'timestamp',
 'source',
 'text',
 'rating_numerator',
 'rating_denominator',
 'name',
 'doggo',
 'floof',
 'pupper',
 'puppo']

### Issue #5: Columns have inconsistent case, invalid names. Columns observed: name, doggo, floof, pupper, puppo. Also fix case p1, p2, p3 in image_predictions.

#### Define
- Keep the 'None' names, and make the 'a' names 'None' as well.
- Convert all 'None" to Nan
- Change case to capitalize

#### Code

In [865]:
# Change the dog names that are 'a' to 'None'
twitter_archive_clean['name'] = twitter_archive_clean.name.replace('a', 'None')

# Change all columns that contain 'None' to NaN
cols = ['name','doggo','floof','pupper','puppo']
twitter_archive_clean[cols] = twitter_archive_clean[cols].replace('None', np.nan)

# Fix the case issues on cols by using str.capitalize()
twitter_archive_clean[cols]= twitter_archive_clean[cols].apply(lambda x: x.astype(str).str.capitalize())

cols2 = ['p1', 'p2', 'p3']
image_pred_clean[cols2] = image_pred_clean[cols2].apply(lambda x: x.astype(str).str.capitalize())


# https://www.datasciencelearner.com/convert-entire-dataframe-columns-lower-case-upper-case/

#### Test

In [866]:
# Validate that names have been capitalized
twitter_archive_clean[cols].sample(15), image_pred_clean[cols2].sample(15)

(          name  doggo floof pupper  puppo
 1731    Willie    Nan   Nan    Nan    Nan
 684        Nan    Nan   Nan    Nan    Nan
 508        Nan    Nan   Nan    Nan    Nan
 1667      Joey    Nan   Nan    Nan    Nan
 2235       Nan    Nan   Nan    Nan    Nan
 1407    Reagan    Nan   Nan    Nan    Nan
 493        Nan  Doggo   Nan    Nan    Nan
 1462    Archie    Nan   Nan    Nan    Nan
 1347  Bisquick    Nan   Nan    Nan    Nan
 1352       Nan    Nan   Nan    Nan    Nan
 1481     Sadie    Nan   Nan    Nan    Nan
 2067       Nan    Nan   Nan    Nan    Nan
 324     Lipton    Nan   Nan    Nan    Nan
 961     Cooper    Nan   Nan    Nan  Puppo
 1745       Nan    Nan   Nan    Nan    Nan,
                      p1                        p2                      p3
 127            Pembroke                  Cardigan                 Basenji
 1176           Pembroke                  Cardigan               Chihuahua
 855          Car_mirror  Chesapeake_bay_retriever                  Vizsla
 1631      

### Issue #6: Values above and below 10 observed in rating_denominator column.

#### Define
- Identified multiple rows where the denominator had a value that was not 10. Adjust denominators where appropriate, and drop rows where no rating is possible.

In [867]:
# Query the clean archive to find remaining rows to be fixed
twitter_archive_clean.query('rating_denominator != 10')[['text', 'rating_numerator', 'rating_denominator']]

,text,rating_numerator,rating_denominator
433,The floofs have been released I repeat the floofs have been released. 84/70 https://t.co/NIYC820tmd,84,70
516,Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\nhttps://t.co/98tB8y7y7t https://t.co/LouL5vdvxx,24,7
902,Why does this never happen at my front door... 165/150 https://t.co/HmwrdfEfUE,165,150
1068,"After so many requests, this is Bretagne. She was the last surviving 9/11 search dog, and our second ever 14/10. RIP https://t.co/XAVDNDaVgQ",9,11
1120,Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once https://t.co/yGQI3He3xv,204,170
1165,Happy 4/20 from the squad! 13/10 for all https://t.co/eV1diwds8a,4,20
1202,This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10 https://t.co/Kky1DPG4iq,50,50
1228,Happy Saturday here's 9 puppers on a bench. 99/90 good work everybody https://t.co/mpvaVxKmc1,99,90
1254,Here's a brigade of puppers. All look very prepared for whatever happens next. 80/80 https://t.co/0eb7R1Om12,80,80
1274,"From left to right:\nCletus, Jerome, Alejandro, Burp, &amp; Titson\nNone know where camera is. 45/50 would hug all at once https://t.co/sedre1ivTK",45,50


In [868]:
# Rows to Fix
idx_fix_den = [1068, 1165, 1662, 2335]
#idx_fix_num = {313 : 13, 784 : 14, 1068 : 14, 1165 : 13, 1202 : 11, 1662 : 10, 2335 : 9}
idx_fix_num = [[1068, 14], [1165, 13], [1662, 10], [2335, 9]]
#
# Rows to Delete
idx_del = [433, 516, 902, 1120, 1202, 1228, 1254, 1274, 1351, 1433, 1635, 1779, 1843]

# Set the denominators = 10
for i in idx_fix_den:
    twitter_archive_clean.at[i, 'rating_denominator'] = 10

# Set the numerators = new value
for i, v in idx_fix_num:
    twitter_archive_clean.at[i, 'rating_numerator'] = v

# Drop rows
twitter_archive_clean.drop(index=idx_del, inplace=True)

#### Test

In [869]:
# Expect to have zero rows
twitter_archive_clean.query('rating_denominator != 10')[['text', 'rating_numerator', 'rating_denominator']]

,text,rating_numerator,rating_denominator


### Issue #7: Rename column to tweet_id in twitter_details table

#### Define
- In the tweet_details_clean table, change the column name from id to tweet_id

In [870]:
# Rename the column
tweet_details_clean.rename(columns={'id' : 'tweet_id'}, inplace=True)

#### Test

In [871]:
# Display first 5 rows and validate column name has changed
tweet_details_clean.head()

,tweet_id,retweet_count,favorite_count
0,892420643555336193,8853,39467
1,892177421306343426,6514,33819
2,891815181378084864,4328,25461
3,891689557279858688,8964,42908
4,891327558926688256,9774,41048


### Issue #8: Non-Dog names in columns p1, p2, p3

#### Define
- Identify p1, p2 and p3 values that have names for images that are not dogs. Drop these rows.

#### Code

In [872]:
# Create a unique list of names to identify non-dog names.
p1 = np.array(image_pred_clean.p1.to_list())
p1_unique = np.unique(p1)
print(p1_unique)

['Acorn_squash' 'Afghan_hound' 'African_crocodile' 'African_grey'
 'African_hunting_dog' 'Agama' 'Airedale' 'Alp' 'American_black_bear'
 'American_staffordshire_terrier' 'Angora' 'Appenzeller' 'Arabian_camel'
 'Arctic_fox' 'Australian_terrier' 'Axolotl' 'Badger' 'Bakery'
 'Bald_eagle' 'Balloon' 'Banana' 'Bannister' 'Barbell' 'Barrow' 'Basenji'
 'Basketball' 'Basset' 'Bath_towel' 'Bathtub' 'Beach_wagon' 'Beagle'
 'Bearskin' 'Beaver' 'Bedlington_terrier' 'Bee_eater'
 'Bernese_mountain_dog' 'Bib' 'Bighorn' 'Binoculars' 'Birdhouse' 'Bison'
 'Black-and-tan_coonhound' 'Black-footed_ferret' 'Blenheim_spaniel'
 'Bloodhound' 'Bluetick' 'Boathouse' 'Bonnet' 'Book_jacket' 'Bookcase'
 'Bookshop' 'Border_collie' 'Border_terrier' 'Borzoi' 'Boston_bull' 'Bow'
 'Bow_tie' 'Box_turtle' 'Boxer' 'Brabancon_griffon' 'Briard'
 'Brittany_spaniel' 'Brown_bear' 'Bubble' 'Bull_mastiff' 'Bustard' 'Cairn'
 'Candle' 'Canoe' 'Car_mirror' 'Cardigan' 'Carousel' 'Carton'
 'Cash_machine' 'Cheeseburger' 'Cheetah' 'Chesa

In [873]:
# Create list of non-log names to drop
to_drop = ['Acorn_squash', 'African_crocodile', 'American_black_bear', 'Arabian_camel', 'Arctic_fox', 'Badger', 'Bakery',
           'Bald_eagle', 'Balloon', 'Banana', 'Bannister', 'Barbell', 'Basketball', 'Bath_towel', 'Bathtub', 'Beach_wagon', 
           'Bearskin' 'Beaver', 'Bee_eater', 'Bib', 'Bighorn', 'Binoculars', 'Birdhouse', 'Bison', 'Black-footed_ferret', 
           'Boathouse', 'Bonnet', 'Book_jacket', 'Bookcase', 'Bookshop', 'Bow', 'Bow_tie' 'Box_turtle', 'Brown_bear', 'Bubble', 
           'Candle', 'Canoe', 'Car_mirror', 'Cardigan', 'Carousel', 'Carton', 'Cash_machine', 'Cheeseburger', 'Cheetah', 
           'Chimpanzee', 'China_cabinet', 'Christmas_stocking', 'Cliff', 'Clog', 'Coffee_mug,' 'Coho', 'Coil', 'Comic_book', 
           'Common_iguana', 'Conch', 'Convertible', 'Coral_reef', 'Cougar', 'Cowboy_boot', 'Cowboy_hat', 'Crane', 'Crash_helmet', 
           'Cuirass', 'Cup', 'Desktop_computer', 'Dhole', 'Dingo', 'Dining_table', 'Dishwasher', 'Dogsled', 'Doormat', 'Dough', 
           'Earthstar', 'Egyptian_cat', 'Electric_fan', 'Envelope', 'Espresso', 'Feather_boa', 'Fiddler_crab', 'Fire_engine', 'Flamingo', 
           'Fountain', 'Four-poster', 'Frilled_lizard', 'Gas_pump', 'Geyser', 'Giant_panda', 'Gondola', 'Goose', 'Grey_fox', 
           'Guinea_pig', 'Hammer', 'Hamster', 'Hand_blower', 'Handkerchief', 'Hare', 'Harp', 'Hay', 'Hen', 'Hermit_crab', 'Hippopotamus', 
           'Hog', 'Home_theater', 'Hotdog', 'Hummingbird', 'Hyena', 'Ice_bear', 'Ice_lolly', "Jack-o'-lantern", 'Jellyfish', 'Jigsaw_puzzle', 
           'Killer_whale', 'King_penguin', 'Koala', 'Komondor', 'Lakeside', 'Laptop', 'Lawn_mower', 'Leaf_beetle', 'Leatherback_turtle', 
           'Limousine', 'Lion', 'Llama', 'Loafer', 'Long-horned_beetle', 'Lorikeet', 'Lynx', 'Madagascar_cat', 'Mailbox', 'Meerkat' 
           'Microphone', 'Microwave', 'Military_uniform', 'Minibus', 'Minivan', 'Mortarboard', 'Motor_scooter', 'Mousetrap', 'Mud_turtle', 
           'Muzzle', 'Nail', 'Orange', 'Ostrich', 'Otter', 'Ox', 'Paddle', 'Padlock', 'Panpipe', 'Paper_towel', 'Park_bench', 'Patio', 
           'Peacock', 'Pedestal', 'Pencil_box', 'Picket_fence', 'Piggy_bank', 'Pillow', 'Ping-pong_ball', 'Pitcher', 'Platypus', 'Pole', 
           'Polecat', 'Pool_table', 'Porcupine', 'Pot', 'Prayer_rug', 'Prison', 'Quilt', 'Radio_telescope', 'Rain_barrel', 'Ram', 'Rapeseed', 
           'Refrigerator', 'Remote_control', 'Restaurant', 'Revolver', 'Robin', 'Rotisserie', 'Sandbar', 'School_bus', 'Scorpion', 
           'Sea_lion', 'Sea_urchin', 'Seashore', 'Seat_belt', 'Shield', 'Shopping_basket', 'Shopping_cart', 'Shower_curtain', 'Siamese_cat', 
           'Ski_mask', 'Skunk', 'Sliding_door', 'Slug', 'Snail', 'Snorkel', 'Snowmobile', 'Soccer_ball', 'Space_heater', 'Starfish', 
           'Stone_wall', 'Stove', 'Street_sign', 'Studio_couch', 'Suit', 'Sulphur-crested_cockatoo', 'Sundial', 'Sunglasses', 'Sussex_spaniel', 
           'Swab', 'Swing', 'Syringe', 'Tabby', 'Tailed_frog', 'Teapot', 'Tennis_ball', 'Terrapin', 'Three-toed_sloth', 'Tick', 'Tiger_shark', 
           'Timber_wolf', 'Toilet_seat', 'Toilet_tissue', 'Toyshop', 'Traffic_light', 'Triceratops', 'Tricycle', 'Trombone', 'Tub', 'Upright', 
           'Vacuum', 'Walking_stick', 'Wallaby', 'Washbasin', 'Washer', 'Water_bottle', 'Water_buffalo', 'Weasel', 'Web_site', 'White_wolf', 
           'Wild_boar', 'Window_shade', 'Wombat', 'Wood_rabbit', 'Wooden_spoon', 'Wool', 'Zebra']

image_pred_clean = image_pred_clean[~image_pred_clean.p1.isin(to_drop)]
image_pred_clean = image_pred_clean[~image_pred_clean.p2.isin(to_drop)]
image_pred_clean = image_pred_clean[~image_pred_clean.p2.isin(to_drop)]

#### Test

In [876]:
# Started with 2075 rows.
image_pred_clean.p1.sample(10), image_pred_clean.shape

(1306                        Whippet
 1340       Chesapeake_bay_retriever
 852                      Pomeranian
 1642               Golden_retriever
 1454                      Chihuahua
 504     West_highland_white_terrier
 726                             Pug
 1790                     Toy_poodle
 1523                           Chow
 1114    German_short-haired_pointer
 Name: p1, dtype: object,
 (1378, 12))

## Tidy Issues
### Issue #1: Dog varibles doggo, floofer (floof), pupper, and puppo are in individual columns and should be categorical

#### Define
- The four dog stage columns in twitter_archive_clean need to be converted into a single categorical column called dog_stage. Using the Pandas Melt function, they will be turned into rows. The temporary variable column will be dropped and then the duplicate rows that result from the melt are dropped with an index reset.

In [780]:
# List of columns to 'melt' around
cols = ['tweet_id', 'timestamp', 'source', 'text', 'rating_numerator', 'rating_denominator', 'name']

twitter_archive_clean = twitter_archive_clean.melt(id_vars=cols, value_name='dog_stage').drop('variable', axis=1)
twitter_archive_clean = twitter_archive_clean[twitter_archive_clean.dog_stage != ''].drop_duplicates().reset_index(drop=True)

#### Test

In [781]:
# Validate that the new dog_stage column is present and contains values
twitter_archive_clean.head()

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,dog_stage
0,892420643555336193,2017-08-01 16:23:56+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU,13,10,Phineas,Nan
1,892177421306343426,2017-08-01 00:17:27+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV",13,10,Tilly,Nan
2,891815181378084864,2017-07-31 00:18:03+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10 https://t.co/wUnZnhtVJB,12,10,Archie,Nan
3,891689557279858688,2017-07-30 15:58:51+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ,13,10,Darla,Nan
4,891327558926688256,2017-07-29 16:00:24+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","This is Franklin. He would like you to stop calling him ""cute."" He is a very fierce shark and should be respected as such. 12/10 #BarkWeek https://t.co/AtUZn91f7f",12,10,Franklin,Nan


### Issue #2: Columns from image_predictions and tweet_details should merge with twitter_archive

#### Define
- Merge columns from twitter_archive_clean with the retweet_count and favorite_count from the tweet_details_clean table. Then, bring in the image_pred_clean columns.

In [743]:
# Merge columns from tweet_details_clean and then image_pred_clean
twitter_master = pd.merge(left=twitter_archive_clean, right=tweet_details_clean, how='left', on='tweet_id')
twitter_master = pd.merge(left=twitter_master, right=image_pred_clean, how='left', on='tweet_id')

#### Test

In [744]:
# Validate that all the columns with data are present
twitter_master.sample(10)

,tweet_id,timestamp,source,text,rating_numerator,rating_denominator,name,dog_stage,retweet_count,favorite_count,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1726,672640509974827008,2015-12-04 04:56:09+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Leonidas. He just got rekt by a snowball. 9/10 doggy down https://t.co/uNrmYDUa9M,9,10,Leonidas,Nan,348,1032,https://pbs.twimg.com/media/CVWy9v-VAAALSoE.jpg,1.0,Chesapeake_bay_retriever,0.420155,True,Cardigan,0.266030,True,Labrador_retriever,0.042514,True
616,772152991789019136,2016-09-03 19:23:13+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Here's a couple rufferees making sure all the sports are played fairly today. Both 10/10 would bribe with extra pets https://t.co/H9yjI9eo3A,10,10,Nan,Nan,1300,4181,https://pbs.twimg.com/media/Crc9DEoWEAE7RLH.jpg,2.0,Golden_retriever,0.275318,True,Irish_setter,0.100988,True,Vizsla,0.073525,True
1922,668955713004314625,2015-11-24 00:54:05+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is a Slovakian Helter Skelter Feta named Leroi. Likes to skip on roofs. Good traction. Much balance. 10/10 wow! https://t.co/Dmy2mY2Qj5,10,10,Nan,Nan,77,300,https://pbs.twimg.com/media/CUibq3uVAAAup_O.jpg,1.0,Cocker_spaniel,0.367492,True,Lakeland_terrier,0.272621,True,Soft-coated_wheaten_terrier,0.067006,True
1991,667546741521195010,2015-11-20 03:35:20+00:00,"<a href=""http://twitter.com"" rel=""nofollow"">Twitter Web Client</a>",Here is George. George took a selfie of his new man bun and that is downright epic. (Also looks like Rand Paul) 9/10 https://t.co/afRtVsoIIb,9,10,George,Nan,138,355,https://pbs.twimg.com/media/CUOaOWXWcAA0_Jy.jpg,1.0,Toy_poodle,0.787424,True,Miniature_poodle,0.202225,True,Teddy,0.004047,False
1623,675006312288268288,2015-12-10 17:37:00+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",Say hello to Mollie. This pic was taken after she bet all her toys on Ronda Rousey. 10/10 hang in there pupper https://t.co/QMmAqA9VqO,10,10,Mollie,Nan,252,1057,https://pbs.twimg.com/media/CV4aqCwWsAIi3OP.jpg,1.0,Boxer,0.654697,True,Space_heater,0.043389,False,Beagle,0.042848,True
816,745074613265149952,2016-06-21 02:03:25+00:00,"<a href=""http://vine.co"" rel=""nofollow"">Vine - Make a Scene</a>",This is Jeffrey. He wasn't prepared to execute such advanced barkour. Still 11/10 would totally pet https://t.co/MuuwkkLrHh,11,10,Jeffrey,Nan,3976,8605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1252,692901601640583168,2016-01-29 02:46:29+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>","""Fuck the system"" 10/10 https://t.co/N0OADmCnVV",10,10,Nan,Nan,682,1971,https://pbs.twimg.com/media/CZ2uU37UcAANzmK.jpg,1.0,Soft-coated_wheaten_terrier,0.403496,True,Cocker_spaniel,0.135164,True,Golden_retriever,0.088719,True
507,789986466051088384,2016-10-23 00:27:05+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Happy. He's a bathtub reviewer. Seems to be pleased with this one. 12/10 https://t.co/Ln89R4FP7v,12,10,Happy,Nan,2704,10369,https://pbs.twimg.com/media/CvaYgDOWgAEfjls.jpg,1.0,Tub,0.479477,False,Bathtub,0.325106,False,Golden_retriever,0.078530,True
1422,682406705142087680,2015-12-31 03:43:31+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Patrick. He's a bigass pupper. 7/10 https://t.co/J9DXBFoAQe,7,10,Patrick,Nan,2019,8319,https://pbs.twimg.com/media/CXhlRmRUMAIYoFO.jpg,1.0,Wombat,0.709344,False,Koala,0.169758,False,Beaver,0.079433,False
2215,776477788987613185,2016-09-15 17:48:25+00:00,"<a href=""http://twitter.com/download/iphone"" rel=""nofollow"">Twitter for iPhone</a>",This is Huck. He's addicted to caffeine. Hope it's not too latte to seek help. 11/10 stay strong pupper https://t.co/iJE

## Storing Data
Save gathered, assessed, and cleaned master dataset to a CSV file named "twitter_archive_master.csv".

In [742]:
# Save twitter_master to csv file with no index
twitter_master.to_csv('twitter_archive_master.csv', index=False)

## Analyzing and Visualizing Data
In this section, analyze and visualize your wrangled data. You must produce at least **three (3) insights and one (1) visualization.**

### Insights:
1.

2.

3.

### Visualization

### References
Downloading files using requests library: https://www.tutorialspoint.com/downloading-files-from-web-using-python<br>
Read Json file line by line into dataframe: https://stackoverflow.com/questions/20037430/reading-multiple-json-records-into-a-pandas-dataframe<br>
Strings that do not contain a value: https://stackoverflow.com/questions/17097643/search-for-does-not-contain-on-a-dataframe-in-pandas<br>
Convert case on multiple columns at same time: https://www.datasciencelearner.com/convert-entire-dataframe-columns-lower-case-upper-case/<br>

